# Accessing ICESat-2 Data Using icepyx
This notebook illustrates the use of [icepyx](https://icepyx.readthedocs.io/en/latest/) for programmatic ICESat-2 data query and download from the NASA NSIDC DAAC (NASA National Snow and Ice Data Center Distributed Active Archive Center).
A [similar data access notebook](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_data_access.html) provides more details on inputs and user options, and a complimentary notebook demonstrates in greater detail the [subsetting](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_data_access2-subsetting.html) options available when ordering data.

## Motivation and Background

`icepyx` is a wrapper on top of the NSIDC API, originally created to make programmatic access to ICESat-2 data easier. The library is open-source, community developed and driven, and was created in response to user challenges faced at the 2019 ICESat-2 Cryosphere-themed Hackweek. At that time, the EarthData library (link to other tutorial here) had not been developed, and there was a steep learning curve for programmatic access to ICESat-2 data.

The EarthData and icepyx developers are working together to leverage the strengths of each library, ensure compatibility, and reduce duplication of effort by sharing infrastructure (e.g. Earthdata authentication for data access).

:::{figure-md} ED-icepyx checklist
<img src="https://drive.google.com/file/d/13MtWKhY12abQRGdNXTUD61CGRuG3oYxH/view?usp=sharing" alt="Earthdata-icepyx comparison checklist" width="800px">
Checklist illustrating the relative strengths of the icepyx and the earthdata libraries.
:::

Under continuous development, we think you'll find `icepyx` a useful resource and encourage you to contribute your ideas, code, and feedback to enhance its use throughout your entire ICESat-2 workflow. In addition to the programmatic data access presented here, icepyx also includes modules for loading data into memory (with plans for lazy loading via Dask), cloud data access (via a soon-to-be familiar interface), working with ICESat-2's nested data variables, and templates for adding other data sources (QUEST - Query Unify Explore SpatioTemporal; currently finalizing the integration of Argo data as our first example case). We hope that this year's Hackweek continues to expand our ICESat-2 data-user community and sparks future feature development, both through hacking projects and beyond.

## Computing environment

We'll be using the following open source Python libraries in this notebook:

In [ ]:
import icepyx as ipx
%matplotlib inline

## Key Steps for Programmatic Data Access

There are several key steps for accessing data from the NSIDC API:
1. Define your parameters (spatial, temporal, dataset, etc.)
2. Query the NSIDC API to find out more information about the dataset
3. Log in to NASA Earthdata
4. Define additional parameters (e.g. subsetting/customization options)
5. Order your data
6. Download your data

icepyx streamlines this process into a minimal number of lines of code.

### Step 1: Create an ICESat-2 data object with the desired search parameters

There are three required inputs, depending on how you want to search for data. Two are required in all cases:
- `short_name` = the data product of interest, known as its "short name".
See https://nsidc.org/data/icesat-2/data-sets for a list of the available data products.
- `spatial extent` = a region of interest to search within. This can be entered as a bounding box, polygon vertex coordinate pairs, or a polygon geospatial file (currently shp, kml, and gpkg are supported).
    - bounding box: Given in decimal degrees for the lower left longitude, lower left latitude, upper right longitude, and upper right latitude
    - polygon vertices: Given as longitude, latitude coordinate pairs of decimal degrees with the last entry a repeat of the first.
    - polygon file: A string containing the full file path and name.
    

Then, for all non-gridded products (ATL<=13), you must include AT LEAST one of the following inputs (temporal or orbital constraints):
- `date_range` = the date range for which you would like to search for results. Must be formatted as a set of 'YYYY-MM-DD' strings separated by a comma.
- `cycles` = Which orbital cycle to use, input as a numerical string or a list of strings. If no input is given, this value defaults to all available cycles within the search parameters.  An orbital cycle refers to the 91-day repeat period of the ICESat-2 orbit.
- `tracks` = Which [Reference Ground Track (RGT)](https://icesat-2.gsfc.nasa.gov/science/specs) to use, input as a numerical string or a list of strings. If no input is given, this value defaults to all available RGTs within the spatial and temporal search parameters.

<div class="alert alert-block alert-info">
<b>Warning:</b> Although you can use cycles and tracks to identify available granules, unfortunately you will not be able to order this specific list of granules using `ipx.order_granules()`. This is because NSIDC cannot handle orders for granules filtered by cycle and track. As of March 2022, NSIDC is working to implement this capability, after which time we can enable this functionality in icepyx.</div>

Here we will use a bounding box example. Please see the [icepyx documentation](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_data_access.html) for examples using other `spatial_extent` types or to search for granules using orbital parameters.

In [ ]:
# bounding box
short_name = 'ATL06'
spatial_extent = [-55, 68, -48, 71]
date_range = ['2019-02-20','2019-02-28']

Create the data object using our inputs

In [ ]:
region_a = ipx.Query(short_name, spatial_extent, date_range)

Formatted parameters and function calls allow us to see the the properties of the data object we have created.

In [ ]:
print(region_a.product)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.cycles)
print(region_a.tracks)
print(region_a.product_version)
region_a.visualize_spatial_extent()

There are also several optional inputs to allow the user finer control over their search. Start and end time are only valid inputs on a temporally limited search.
- `start_time` = start time to search for data on the start date. If no input is given, this defaults to 00:00:00.
- `end_time` = end time for the end date of the temporal search parameter. If no input is given, this defaults to 23:59:59. Times must be input as 'HH:mm:ss' strings.
- `version` = What version of the data product to use, input as a numerical string. If no input is given, this value defaults to the most recent version of the product specified in `short_name`.

*NOTE It is best to use the most recent version where possible. If you try to order/download too old a version (such that it is no longer hosted by NSIDC), you will get a "no data matched your request" error.*

### More information about your query object
In addition to viewing the stored object information shown above (e.g. product short name, start and end date and time, version, etc.), we can also request summary information about the data product itself or confirm that we have manually specified the latest version.

In [ ]:
region_a.product_summary_info()
print(region_a.latest_version())

If the summary does not provide all of the information you are looking for, or you would like to see information for previous versions of the data product, all available metadata for the collection product is available in a readable format.

In [ ]:
region_a.product_all_info()

### Step 2: Querying a data product

In order to search the product collection for available data granules, we need to build our search parameters. This is done automatically behind the scenes when you run `region_a.avail_granules()`, but you can also build and view them by calling `region_a.CMRparams`. These are formatted as a dictionary of key:value pairs according to the [CMR documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html) and are analogous to the methods called with an `earthdata.DataGranules()` instance.

In [ ]:
#build and view the parameters that will be submitted in our query
region_a.CMRparams

Now that our parameter dictionary is constructed, we can search the CMR database for the available granules.
Granules returned by the CMR metadata search are automatically stored within the data object.
The search completed at this level relies completely on the granules' metadata.
As a result, some (and in rare cases all) of the granules returned may not actually contain data in your specified region, particularly if the region is small or located near the boundaries of a given granule. If this is the case, the subsetter will not return any data when you actually place the order.
A warning message will be issued during ordering for each granule to which this applies (but no message is output for successfully subsetted granules, so don't worry!)

In [ ]:
#search for available granules and provide basic summary info about them
region_a.avail_granules()

In [ ]:
#get a list of granule IDs for the available granules
region_a.avail_granules(ids=True)

In [ ]:
#print detailed information about the returned search results
region_a.granules.avail

### Step 3: Log in to NASA Earthdata
In order to download any data from NSIDC, we must first authenticate ourselves using a valid Earthdata login. This will create a valid token to interface with the DAAC as well as start an active logged-in session to enable data download. Once you have successfully logged in for a given query instance, the token and session will be passed behind the scenes as needed for you to order and download data. Passwords are entered but not shown or stored in plain text by the system.

There are multiple ways to provide your Earthdata credentials via icepyx.
The in-notebook login is used below, and other methods (via a saved environment variable (hub-appropriate) or .netrc file (recommended for local use only)) will soon be detailed in the documentation.

In [ ]:
earthdata_uid = 'icepyx_devteam'
email = 'icepyx.dev@gmail.com'
region_a.earthdata_login(earthdata_uid, email)

### Step 4: Additional Parameters and Subsetting

Once we have generated our session, we must build the required configuration parameters needed to actually download data. These will tell the system how we want to download the data. As with the CMR search parameters, these will be built automatically when you run `region_a.order_granules()`, but you can also create, view, and manually update them with `region_a.reqparams`.

#### Don't we already have our granules?

Technically, yes. By searching CMR, we used metadata to quickly determine which granules likely contain information we'd like. But to find your favorite quote in your favorite book, you actually have to open the book. CMR just helped us figure out where in the library it was and get it off the shelf. Subsetting allows us to just check out and use that quote instead of the whole book.

:::{figure-md} Filter versus subset visual
<img src="https://drive.google.com/file/d/15TwUrLEepzvW-GN3qP7FYKIHCIkk5v1X/view?usp=sharing" alt="Filter-vs-subset maps with code for each step in earthdata and icepyx" width="800px">
Illustration of the steps and code (in icepyx and earthdata) required to filter data in CMR using spatial bounds, followed by the pre-processing step of subsetting offered as a service by NSIDC.
:::

#### Subsetting

In addition to the required parameters (CMRparams and reqparams) that are submitted with our order, for ICESat-2 data products we can also submit subsetting parameters to NSIDC.
For a deeper dive into subsetting and the reformatting options (e.g. file format conversion) options offered by NSIDC, please see our [Subsetting Tutorial Notebook](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_data_access2-subsetting.html), which covers subsetting in more detail, including how to get a list of subsetting options, how to build your list of subsetting parameters, and how to generate a list of desired variables (most datasets have more than 200 variable fields!), including using pre-built default lists (these lists are still in progress and we welcome contributions!).

Subsetting utilizes the NSIDC's built in subsetter to extract only the data you are interested (spatially, temporally, variables of interest, etc.). The advantages of using the NSIDC's subsetter include:
* easily reproducible downloads, particularly when coupled with an icepyx query object
* smaller file size, meaning faster downloads, less storage required, and no need to subset the data on your own
* still easy to go back and order more data/variables with the same or similar search parameters
* no extraneous data means you can move directly to analysis and easily navigate your dataset

Certain subset parameters are specified by default unless `subset=False` is included as an input to `order_granules()` or `download_granules()` (which calls `order_granules()` under the hood).

As for the CMR and required parameters, default subset parameters can be built and viewed using `subsetparams`. Where an input spatial file is used, rather than a bounding box or manually entered polygon, the spatial file will be used for subsetting (unless subset is set to False) but not show up in the `subsetparams` dictionary.

In [ ]:
region_a.subsetparams()

### Step 5: Place the order
Then, we can send the order to NSIDC using the order_granules function. Information about the granules ordered and their status will be printed automatically. Status information can also be emailed to the address provided when the `email` kwarg is set to `True`. Additional information on the order, including request URLs, can be viewed by setting the optional keyword input 'verbose' to True.

In [ ]:
region_a.order_granules()

In [ ]:
#view a short list of order IDs
region_a.granules.orderIDs

### Step 6: Download the order
Finally, we can download our order to a specified directory (which needs to have a full path but doesn't have to point to an existing directory) and the download status will be printed as the program runs. Additional information is again available by using the optional boolean keyword `verbose`.

In [ ]:
path = './download'
region_a.download_granules(path)

---------------------------------

## Quick-Start Guide

The entire process of getting ICESat-2 data (from query to download) can ultimately be accomplished in three minimal lines of code:

`reg_a = ipx.Query(short_name, spatial_extent, date_range)`

`reg_a.earthdata_login(earthdata_uid, email)`

`reg_a.download_granules(path)`

where the function inputs are as described above.
The steps detailed in this tutorial are meant to give the user full control over the data they find and download while providing a higher-level interface and realistic defaults suitable for most use cases.


---------------------------------

## Quick-Start Guide (Cloud Data Access)

If you want to access ICESat-2 data in the cloud with icepyx, you need to turn on a few flags and activate your credentials for cloud access:

```
import s3fs
import h5py

reg_a = ipx.Query(short_name, spatial_extent, date_range)
gran_ids = reg_a.avail_granules(ids=True, s3urls=True)


reg_a.earthdata_login(earthdata_uid, email, s3token=True)
credentials = reg._s3login_credentials

s3 = s3fs.S3FileSystem(key=credentials['accessKeyId'],
                       secret=credentials['secretAccessKey'],
                       token=credentials['sessionToken'])

# select a granule to read
s3url = gran_ids[1][0]

f = h5py.File(s3.open(s3url,'rb'),'r')
```

where the function inputs are as described above.
A full tutorial on cloud data access via icepyx is [here](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_cloud_data_access.html).


________________________________

## Other Resources

Links to more detailed examples using icepyx:
* 
*

ICESat-2 Resources Guide...


**Credits**

MOVE TO TOP?

* based on the [icepyx ICESat-2 Data Access Notebook](https://icepyx.readthedocs.io/en/latest/example_notebooks/IS2_data_access.html) and source materials therein.
* notebook contributors (this tutorial): Mikala Beig, Jessica Scheick
* additional contributors (source materials): Amy Steiker, Tyler Sutterley, Bruce Wallin